In [1]:
!pip install geojson

In [2]:
import geojson
import requests
fname = "SA_RoadsExample_lines.geojson"
geojson_data = None
with open(fname) as json_file:
    geojson_data = geojson.load(json_file)
    

In [3]:
#draw the geometry for the feature on a map
import ipyleaflet as ipy 
import ipywidgets as ipyw
from ipyleaflet import GeoJSON, Map, Marker

x_coord = -34.74
y_coord = 138.78

map1 = ipy.Map(center=[x_coord, y_coord], zoom=12)
label = ipyw.Label(layout=ipyw.Layout(width='100%'))

geo_json1 = GeoJSON(data=geojson_data, style = {'color': 'blue', 'opacity':0.8, 'weight':1.9, 'fillOpacity':0.3})
map1.add_layer(geo_json1)
map1

Map(center=[-34.74, 138.78], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom…

In [4]:
dggs_api_param = {
    'resolution' : 8,
    "dggs_as_polygon" : False
}

In [6]:
import requests
DGGS_WEB_API_URL = "https://dggs.loci.cat/api/search/find_dggs_by_geojson"
r = requests.post(DGGS_WEB_API_URL, params=dggs_api_param, json=geojson_data)
print(r.status_code)
res = r.json()

200


In [7]:
res

{'meta': {'cells_count': 41},
 'dggs_cells': ['R77512152',
  'R77512240',
  'R77512184',
  'R77512185',
  'R77512238',
  'R77512246',
  'R77512271',
  'R77512262',
  'R77512265',
  'R77512230',
  'R77512260',
  'R77512236',
  'R77512232',
  'R77512243',
  'R77512264',
  'R77512273',
  'R77512157',
  'R77512263',
  'R77512231',
  'R77512158',
  'R77512266',
  'R77512151',
  'R77512244',
  'R77512182',
  'R77512155',
  'R77512233',
  'R77512270',
  'R77512128',
  'R77512274',
  'R77512235',
  'R77512247',
  'R77512241',
  'R77512234',
  'R77512261',
  'R77512181',
  'R77512206',
  'R77512237',
  'R77512216',
  'R77512217',
  'R77512154',
  'R77512188']}

In [11]:
dggs_api_param = {
    'resolution' : 10,
    "dggs_as_polygon" : True
}

In [12]:
r = requests.post(DGGS_WEB_API_URL, params=dggs_api_param, json=geojson_data)
print(r.status_code)
res = r.json()

200


In [13]:
#draw the geometry for the feature on a map
x_coord = -35.282
y_coord = 149.09


map2 = ipy.Map(center=[x_coord, y_coord], zoom=9)
label = ipyw.Label(layout=ipyw.Layout(width='100%'))

geo_json2 = GeoJSON(data=res, style = {'color': 'blue', 'opacity':0.8, 'weight':1.9, 'fillOpacity':0.3})
map2.add_layer(geo_json2)
map2

Map(center=[-35.282, 149.09], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoo…

In [ ]:
from shapely.geometry import shape, LineString, MultiLineString, Polygon, MultiPolygon
from geojson.utils import coords
from geojson import Feature, FeatureCollection
from auspixdggs.callablemodules.dggs_in_poly_for_geojson_callable import cells_in_poly, get_dggs_cell_bbox
from auspixdggs.callablemodules.dggs_in_line import line_to_DGGS, densify_my_line
#from auspixdggs.callablemodules.dggs_for_lines_geojson_callable import densify_my_line

def bbox(coord_list):
     box = []
     for i in (0,1):
         res = sorted(coord_list, key=lambda x:x[i])
         box.append((res[0][i],res[-1][i]))
     ret = [ box[0][0], box[1][0], box[0][1], box[1][1] ]
     return ret
    
def geojson_to_shape(g):
    return shape(g)
    
def get_cells_in_geojson(geojson, resolution):
    list_cells = []
    for fea in geojson['features']:  # for feature in attribute table
        res_cells = get_cells_in_feature(fea, resolution)
        list_cells = list(list_cells + res_cells)
    return list_cells 

def get_cells_in_feature(fea, resolution, cell_obj=True):
    geom = geojson_to_shape(fea['geometry'])
    curr_coords = list(coords(fea))
    thisbbox = bbox(curr_coords)
    cells = []
    if isinstance(geom, LineString) or isinstance(geom, MultiLineString): 
        fea['geometry']['coordinates'] = densify_my_line(fea['geometry']['coordinates'], resolution)
        curr_coords = list(coords(fea))
        print(curr_coords)        
        res_cells = line_to_DGGS(curr_coords, resolution)  # start at DGGS level 10   
        cells = res_cells
    elif isinstance(geom, Polygon) or  isinstance(geom, MultiPolygon):
        res_cells = cells_in_poly(thisbbox, fea['geometry']['coordinates'], resolution)  # start at DGGS level 10    
        print(res_cells)
        cells = [item[0] for item in res_cells]
    else: #try something anyway
        res_cells = cells_in_poly(thisbbox, fea['geometry']['coordinates'], resolution)  # start at DGGS level 10    
        print(res_cells)
        cells = [item[0] for item in res_cells]
    return cells

     

In [ ]:

DGGS_RESOLUTION=10
list_cells = get_cells_in_geojson(geojson_data, DGGS_RESOLUTION)  
list_features = []
for cell in list_cells:
    bbox_coords = get_dggs_cell_bbox(cell)
    geom_obj = Polygon(bbox_coords)
    feat = Feature(geometry=geom_obj, properties={"dggs_cell_id": str(cell)}) 
    list_features.append(feat)

feature_collection = FeatureCollection(list_features)

#uncomment if you want to get a file output
#geojson_dump = geojson.dumps(feature_collection, indent=4, sort_keys=True)
#with open('poly.geojson', 'w') as json_file:
#    data = geojson.dump(feature_collection, json_file, indent=4, sort_keys=True)   

In [ ]:
map2 = ipy.Map(center=[x_coord, y_coord], zoom=12)
label = ipyw.Label(layout=ipyw.Layout(width='100%'))

geo_json2 = GeoJSON(data=feature_collection, 
                    style={
                          'color': 'black', 
                          'opacity': 1, 
                          'weight':1, 
                          'fillColor': 'red', 
                          'fillOpacity': 0.3})
map2.add_layer(geo_json2)
map2.add_layer(geo_json1)
map2

In [ ]:
import requests
import json
#iterate over features in feature collection and query Loc-I API for features
def find_features_at_dggs_cell(dggs_cell_id):
    payload = {
        'dggs_cell' : dggs_cell_id
    }
    r = requests.get('https://api.loci.cat/api/v1/location/find-at-DGGS-cell', params=payload)
    json_res = r.json()
    return json_res


In [ ]:
target_feature_type = 'asgs16_sa1'

set_of_sa1_features = set([])
for feature in feature_collection['features']:
    cell_id = feature['properties']['dggs_cell_id']
    print("Processing {}".format(cell_id))
    res = find_features_at_dggs_cell(cell_id)
    if 'locations' in res:
        for loc in res['locations']:
            if 'dataType' in loc and loc['dataType'] == target_feature_type:
                set_of_sa1_features.add(loc['uri'])
                print("Adding {}".format(loc['uri']))

    


In [ ]:
print(json.dumps(list(set_of_sa1_features), indent=3))

In [ ]:
def get_geom(loci_uri):
    payload = {
        "uri": loci_uri
    }
    url = "https://api.loci.cat/api/v1/location/geometry"
    r = requests.get(url, params=payload)
    res = r.json()
    #get the first geom result
    geojson_data = []
    if len( res['geometry']) > 0:
        geojson_data = res['geometry'][0]
    return geojson_data




In [ ]:
map3 = ipy.Map(center=[x_coord, y_coord], zoom=12)
label = ipyw.Label(layout=ipyw.Layout(width='100%'))


In [ ]:
for feature_uri in list(set_of_sa1_features):
    curr_geojson_layer = GeoJSON(data=get_geom(feature_uri), 
                    style={
                          'color': 'black', 
                          'opacity': 1, 
                          'weight':1, 
                          'fillColor': 'yellow', 
                          'fillOpacity': 0.3})
    map3.add_layer(curr_geojson_layer)


In [ ]:
map3.add_layer(geo_json1)



In [ ]:
map3